In [1]:
##FROM FILL_UNTAPPD.PY


import os,sys
import pytz
import time
import json
import sqlite3
import requests
import calendar
from dateutil import parser as dateparser


def fillforward():

    requests.packages.urllib3.disable_warnings()

    db_file = os.path.join(os.getcwd(), 'untappd.db')
    conn = sqlite3.connect(db_file)
    c = conn.cursor()

    create = """
    create table if not exists checkins (
        cid INTEGER PRIMARY KEY,
        timestamp INTEGER NOT NULL,
        beer VARCHAR NOT NULL,
        venue VARCHAR NOT NULL,
        json VARCHAR NOT NULL
    )"""

    c.execute(create)

    api_id     = 'E8847D1FA347C37248DEC90E1D21568E18FC5152'
    api_secret = '199C86117D183A9B6BCB3220098F34CFF81FD562'
    brewery_checkin_url = 'https://api.untappd.com/v4/brewery/checkins/451812'

    UTC = pytz.utc
    EST = pytz.timezone('America/New_York')

    url = brewery_checkin_url
    params = {
        'client_id'     : api_id,
        'client_secret' : api_secret,
        'limit'         : 100,
    }


    c.execute('select max(cid) from checkins')
    row = c.fetchone()
    if row and row[0]:
        params['min_id'] = row[0]
    else:
        params['min_id'] = 0
    # if 'min_id' in params:
    #     del params['min_id']
    # params["min_id"] = 0

    while True:
        print(params)
        r = requests.get(url, params=params)
        r.raise_for_status()
        response = r.json()['response']
        checkins = response['checkins']
        max_id = None
        print(checkins)
        for checkin in checkins['items']:
            print (checkin)
            cid = checkin['checkin_id']
            if cid == params['min_id']:
                continue
            elif cid < params['min_id']:
                print('wtf, queried for min_id', params['min_id'], 'and got', cid)
                continue
            if not max_id or cid > max_id: max_id = cid

            when_utc = dateparser.parse(checkin['created_at'])
            timestamp = calendar.timegm(when_utc.timetuple())
            when = EST.normalize(when_utc.astimezone(UTC))
            beer = checkin['beer']['beer_name']
            venue = checkin['venue']['venue_name'] if checkin['venue'] else ''
            checkin_json = json.dumps(checkin)

            
            print(cid, when, beer.encode('utf8'), venue.encode('utf8'))
            c.execute('INSERT OR REPLACE INTO checkins(cid, timestamp, beer, venue, json) values (?, ?, ?, ?, ?)', \
                      (cid, timestamp, beer, venue, checkin_json))
        if max_id is None:
            break
        conn.commit()
        params['min_id'] = max_id
        time.sleep(5)

def backfill():

    c.execute('select min(cid) from checkins')
    row = c.fetchone()
    if row and row[0]:
        params['max_id'] = row[0]

    while True:
        print(params)
        r = requests.get(url, params=params)
        r.raise_for_status()
        response = r.json()['response']
        checkins = response['checkins']
        min_id = None
        for checkin in checkins['items']:
            cid = checkin['checkin_id']
            if params['max_id'] and cid == params['max_id']:
                continue
            if params['max_id'] and cid > params['max_id']:
                continue
            if not min_id or cid < min_id: min_id = cid
            when_utc = dateparser.parse(checkin['created_at'])
            timestamp = int(when_utc.strftime('%s'))
            when = EST.normalize(when_utc.astimezone(UTC))
            beer = checkin['beer']['beer_name']
            venue = checkin['venue']['venue_name'] if checkin['venue'] else ''
            checkin_json = json.dumps(checkin)
            print(cid, when, beer, venue)
            c.execute('INSERT INTO checkins(cid, timestamp, beer, venue, json) values (?, ?, ?, ?, ?)', \
                      (cid, timestamp, beer, venue, checkin_json))
        conn.commit()
        params['max_id'] = min_id
        time.sleep(5)

def get_last_checkin(venue):
     c.execute('select max(timestamp) from checkins where venue=?', (venue,))
     row = c.fetchone()
     return row[0] if row and row[0] else None

def run():
    fillforward()
    
if __name__ == '__main__':
    os.chdir(os.path.abspath(os.path.dirname(sys.argv[0])))

    run()

{'client_id': 'E8847D1FA347C37248DEC90E1D21568E18FC5152', 'client_secret': '199C86117D183A9B6BCB3220098F34CFF81FD562', 'limit': 100, 'min_id': 0}
{'count': 50, 'items': [{'checkin_id': 861531454, 'created_at': 'Sat, 01 Feb 2020 17:29:52 +0000', 'checkin_comment': 'last one :(  send more.', 'rating_score': 4, 'user': {'uid': 7423, 'user_name': 'rader', 'first_name': 'rader', 'last_name': '', 'location': 'back in portland after 10 years in NYC', 'url': '', 'is_supporter': 1, 'bio': 'PGH - PWM - BOS - PWM - NYC - NYS - PWM', 'relationship': None, 'user_avatar': 'https://untappd.akamaized.net/profile/40d3e6a2e4156553dee7faa901a11f6b_100x100.jpg', 'is_private': 0}, 'beer': {'bid': 3589306, 'beer_name': 'Balloon Factory', 'beer_label': 'https://untappd.akamaized.net/site/beer_logos/beer-3589306_c7e6d_sm.jpeg', 'beer_abv': 6.1, 'beer_ibu': 45, 'beer_slug': 'fab-fermentation-arts-fabcans-balloon-factory', 'beer_description': 'Chico Fermented session IPA / pale. West Coast Style. Balanced, clea

{'client_id': 'E8847D1FA347C37248DEC90E1D21568E18FC5152', 'client_secret': '199C86117D183A9B6BCB3220098F34CFF81FD562', 'limit': 100, 'min_id': 861531454}
{'count': 0, 'items': []}


In [2]:
## FROM SYNC_CANS.PY
#!/usr/bin/env python

import fill_untappd
import yaml, json, sqlite3,  dateparser,os, sys
from pprint import pprint
from airtable import Airtable
import pandas as pd


def main():
    fill_untappd.run()
    conn = sqlite3.connect("untappd.db")
    df = pd.read_sql_query("select * from checkins;", conn)
    jsons  = df.json.apply(lambda j: json.loads(j))

    venues = jsons.apply(
        lambda x: 
        pd.concat([
            pd.Series(x["venue"]),
            pd.Series(x["user"])[["user_name"]],
            pd.Series(x["beer"]),
            pd.Series(x)[["checkin_id","created_at","rating_score"]]
            ])
            )

    venues_info = pd.concat([venues, 
    pd.concat([
        venues['contact'].apply(lambda x: pd.Series(x)),
        venues['location'].apply(lambda x: pd.Series(x)),
    ], axis = 1)],axis=1).set_index(["venue_slug","beer_slug"])

    venue_users = venues_info.groupby(["venue_slug","beer_slug"]).user_name.apply(','.join).rename("user_names")
    venue_rating = venues_info.groupby(["venue_slug","beer_slug"]).rating_score.sum().rename("total_rating")
    venue_counts = venues_info.groupby(["venue_slug","beer_slug"]).size().rename("n_checkins").astype(int)
    venue_last_checkin = venues_info.groupby(["venue_slug","beer_slug"]).created_at.apply(lambda x: x.apply(lambda y:dateparser.parse(y)).max()).rename("last_checkin")


    venues_info_dedup = venues_info.loc[~venues_info.index.duplicated()]
    venues_annotated = venues_info_dedup.join(pd.concat([venue_users, venue_rating, venue_counts,venue_last_checkin],axis=1))

    venues_annotated = venues_annotated.loc[ lambda x: x.venue_url != ""]
    venues_annotated = venues_annotated.loc[ venues_annotated.venue_url.notna()]
    venues_annotated = venues_annotated.loc[ lambda x: x.venue_city != ""]
    venues_annotated = venues_annotated.loc[ venues_annotated.venue_city.notna()]


    cans = pd.read_csv("../_data/cans.csv")
    cans["untappd_id"] = cans.untappd.str.extract('(\d*$)').astype(int)

    venue_beers_annotated = venues_annotated.loc[lambda x: x.venue_id.notna()]
    venue_beers_annotated = venue_beers_annotated.join(cans.set_index("untappd_id")[["code"]], on ="bid")
    venue_beers_annotated = venue_beers_annotated.rename({"code":"can_code"}, axis = "columns")
    venue_beers_annotated["venue_id"] = venue_beers_annotated["venue_id"].astype(int)
    venue_beers_annotated.n_checkins = venue_beers_annotated.n_checkins.astype(int)

    venue_beers_annotated.to_csv("../_data/venue_beers.csv")

    venues_annotated = venue_beers_annotated.reset_index()[["venue_name","venue_slug","primary_category","contact","venue_id","lat","lng","venue_city" , "venue_address","venue_country","venue_city",	"venue_country",	"venue_state",'venue_url']]
    venues_annotated.to_csv("../_data/venues.csv")

if __name__ == "__main__":
    print(sys.argv)
    os.chdir(os.path.abspath(os.path.dirname(sys.argv[0])))
    main()

['/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py', '-f', '/Users/ben/Library/Jupyter/runtime/kernel-bda8bf7e-e5a6-4e7c-88c4-41424b81bf91.json']
{'client_id': 'E8847D1FA347C37248DEC90E1D21568E18FC5152', 'client_secret': '199C86117D183A9B6BCB3220098F34CFF81FD562', 'limit': 100, 'min_id': 861531454}
{'count': 0, 'items': []}


FileNotFoundError: [Errno 2] File b'../_data/cans.csv' does not exist: b'../_data/cans.csv'